In [1]:
import numpy as np
import pandas as pd
from utils.kalman import KalmanFilter, position_kf, orientation_kf, congruence_transformation

### Construction of the original spiral

In [2]:
R = 4.125    # Radius of the spiral (8.25 meter diameter, so radius = 4.125 meters)
N = 3.30     # Number of turns
H = 13       # Total height (13 meters)
T = 1300     # Number of steps (adjustable)

# Calculate the spiral points
original_spiral = np.array([[-R * np.cos(i * (2 * np.pi * N) / T),  R * np.sin(i * (2 * np.pi * N) / T), (i / T) * H] for i in range(T)])

### Data Upload

In [3]:
df = pd.read_csv("sensor_data.csv")
df["v_x"] = df["omega_x"]*R
df["v_y"] = df["omega_y"]*R
df["v_z"] = df["omega_z"]*R
df["a_x"] -= df["g_x"]
df["a_y"] -= df["g_y"]
df["a_z"] -= df["g_z"]
df.describe()

,a_x,a_y,a_z,m_x,m_y,m_z,omega_x,omega_y,omega_z,g_x,g_y,g_z,Pressure,Temperature,v_x,v_y,v_z
count,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000,2249.000000
mean,-0.309586,-0.340934,-0.011699,7.683553,8.306203,-42.345780,0.178559,1.721120,-8.249311,-0.607443,-2.130280,9.494522,102719.547750,22.556621,0.736557,7.099622,-34.028407
std,0.657379,0.864585,1.479193,13.114934,14.594034,7.405968,23.946769,18.391705,16.056425,0.690841,0.685081,0.215468,51.203239,0.373446,98.780421,75.865784,66.232755
min,-2.780000,-5.050000,-6.180000,-26.690000,-24.000000,-57.560000,-428.810000,-167.690000,-148.940000,-4.820000,-6.270000,6.720000,102630.020000,21.960000,-1768.841250,-691.721250,-614.377500
25%,-0.720000,-0.830000,-1.130000,-1.870000,-5.000000,-47.500000,-8.940000,-6.310000,-16.870000,-0.960000,-2.590000,9.390000,102674.150000,22.250000,-36.877500,-26.028750,-69.588750
50%,-0.260000,-0.300000,-0.230000,10.500000,10.190000,-43.750000,0.750000,2.620000,-8.310000,-0.660000,-2.040000,9.550000,102720.770000,22.510000,3.093750,10.807500,-34.278750
75%,0.150000,0.190000,1.000000,18.000000,21.370000,-37.380000,10.440000,10.810000,0.130000,-0.330000,-1.620000,9.640000,102763.120000,22.840000,43.065000,44.591250,0.536250
max,2.070000,3.030000,8.210000,40.250000,35.190000,-17.000000,356.440000,171.370000,156.310000,1.600000,-0.050000,9.770000,102805.970000,23.270000,1470.315000,706.901250,644.778750


In the very simple terms, Kalman filter is an optimal estimation algorithm. It is used when we cannot really see or measure something directly. In this very case, we are predicting the somebody's position based on the information on their initial position $(x,y,z)^T$. Based on those we are also measuring $(v_x,v_y,v_z)$.

The Kalman filter works in two steps:

1. **predict**: during this step we are predicting what is going to be the position at the next sample point
2. **update**: compares the estimate from **predict** with the measurement and updates it based on that, also the filter keeps the information on the estimate uncertainty

And we basically have a while loop with those.

    while(walking):
        extimate = predict(next position)
        update(estimate)

# define parameters
- $\bf{F}$: state transition matrix ($n\times n$)
- $\vec{x}$: state vector ($n\times 1$)
- $\bf{P}$: state covariance matrix ($n\times n$)
- $\bf{Q}$: process noise covariance matrix ($n\times n$)
- $\bf{H}$: measurement covariance matrix ($m\times n$)
- $\vec{z}$: measurement vector ($m\times 1$)
- $\bf{K}$: Kalman Gain Matrix ($n\times m$)

The state vector is vector of positions and measurements. 

$$\vec{x}=\begin{pmatrix} x&y&z&\text{orientation}&\text{azimuth}\end{pmatrix}^T$$

The state transition matrix is a description of how our position changes at each sampling instant. This we take from physics, the classic 

$$\begin{cases}x(t)=x_0+v(t)\cdot t\\
a(t)=x_0+v(t)\cdot t + \frac{1}{2}a(t)\cdot t^2\end{cases}$$.

The rest is basically covariance matrices that help us use the information about the noise and environment to optimise the filter. They also change in each iteration.

IMMEDIATELY I APOLOGIZE FOR USING x,y,z TWICE, I'LL FIGURE SOMETHING OUT
ONE USE IS FOR DIRECTIONS, THE OTHER USE IS WITH STATE VECTOR, MEASURED OUTPUT VECTOR, AND INPUT VECTOR

In [5]:
n, m = 3, 1
position_x = position_kf(n,m)
position_y = position_kf(n,m)
position_z = position_kf(n,m)

predicted_x, predicted_y, predicted_z = [], [], []
measurements_x, measurements_y, measurements_z = df['a_x'], df['a_y'], df['a_z']

#for ux, uy, uz in zip(measurements_x, measurements_y, measurements_z):
for ux in measurements_x:
    # flow: read -> predict -> update -> repeat
    position_x.predict(ux, position_x.Q)
    position_x.y = position_x.x[:m]
    predicted_x.append(position_x.y)
    position_x.update(position_x.y, position_x.R)

(3, 1) times (3, 3) times (1, 3)


ValueError: shapes (3,1) and (3,3) not aligned: 1 (dim 1) != 3 (dim 0)

In [6]:
print(position_x.R.shape)

(1,)


In [5]:
print(congruence_transformation(position_x.H, position_x.P))

[[0.0001]]
